In [1]:
import json
import requests
import pandas as pd
import csv
import time
import sqlite3

In [6]:
longitude = -107.3846
latitude = 24.8433

params = {
    'longitude': -107.3846,
    'latitude': 24.8433
}

headers = {
    'Accept': 'application/json'
}

params = {k: v for k, v in params.items() if v is not None}
url = f"https://power.larc.nasa.gov/api/temporal/climatology/point?parameters=SI_EF_TILTED_SURFACE,T2M_MAX,T2M_MIN,EQUIV_NO_SUN_CONSEC_MONTH,T2M&community=RE&longitude={longitude}&latitude={latitude}&format=JSON"

try:
  response = requests.get(url, params=params, headers=headers)
  if response.status_code == 200:
    print (f"OK: {response.status_code}")
    # return response
  elif response.status_code == 401:
    print (f"Unauthorized - check your access token: {response.status_code}")
    # return response
  elif response.status_code == 403:
    print (f"Forbidden - access token may not have proper permissions: {response.status_code}")
    # return response
  elif response.status_code == 404:
    print (f"Location not found: {response.status_code}")
    # return response
  elif response.status_code == 429:
    print (f"Too many requests - rate limit exceeded: {response.status_code}")
    # return response
  elif response.status_code > 429:
    print(f"Your request has failed with status code: {response.status_code}")
    # return response
  else:
    print("ELSEEEE")
    # response.raise_for_status()
    # return response
except requests.exceptions.ConnectionError as error:
  print (f"Request error: {error}")
  # return



OK: 200


In [10]:
print(response.status_code)
print(response.text)
print(response.reason)

200
{"type":"Feature","geometry":{"type":"Point","coordinates":[-107.3846,24.8433,208.36]},"properties":{"parameter":{"T2M":{"JAN":20.26,"FEB":21.12,"MAR":23.25,"APR":25.7,"MAY":28.41,"JUN":30.3,"JUL":28.83,"AUG":28.08,"SEP":27.23,"OCT":26.35,"NOV":23.46,"DEC":20.22,"ANN":25.28},"T2M_MAX":{"JAN":36.72,"FEB":37.88,"MAR":39.97,"APR":43.63,"MAY":43.11,"JUN":45.87,"JUL":42.43,"AUG":38.92,"SEP":38.39,"OCT":38.08,"NOV":38.73,"DEC":35.53,"ANN":45.87},"T2M_MIN":{"JAN":5.94,"FEB":3.69,"MAR":8.94,"APR":13.35,"MAY":14.69,"JUN":19.18,"JUL":22.08,"AUG":21.98,"SEP":18.96,"OCT":14.8,"NOV":10.36,"DEC":5.11,"ANN":3.69},"EQUIV_NO_SUN_CONSEC_MONTH":{"JAN":4.79,"FEB":4.75,"MAR":4.07,"APR":4.67,"MAY":3.14,"JUN":2.32,"JUL":2.14,"AUG":1.87,"SEP":2.05,"OCT":2.85,"NOV":5.26,"DEC":4.11,"ANN":-999.0},"SI_EF_TILTED_SURFACE_HORIZONTAL":{"JAN":4.24,"FEB":5.25,"MAR":6.41,"APR":7.11,"MAY":7.6,"JUN":7.11,"JUL":6.28,"AUG":5.84,"SEP":5.45,"OCT":5.46,"NOV":4.58,"DEC":4.06,"ANN":-999.0},"SI_EF_TILTED_SURFACE_LAT_MINUS15":

In [30]:
res = json.loads(response.text)
df2 = pd.DataFrame({'codigoPostal': pd.Series(dtype='str'), 'variable': pd.Series(dtype='str'), 'dimension': pd.Series(dtype='str'), 'value_type': pd.Series(dtype='str'), 'numeric_value': pd.Series(dtype='float'), 'string_value': pd.Series(dtype='str')})

# df2.loc[len(df2)] = ['a','b','c','d',-40.32,'f']
postalCode = '80060'

if "properties" in res:
  if "parameter" in res["properties"]:
    for key, value in res["properties"]["parameter"].items():
      for dimension, value in value.items():
        lst = []
        lst.append(postalCode)
        lst.append(key)
        lst.append(dimension)
        lst.append("numeric" if isinstance(value, (int, float)) else ("string " if isinstance(value, str) else "ND" ) )
        lst.append(value if isinstance(value, (int, float)) else None)
        lst.append(value if isinstance(value, str) else None)
        df2.loc[len(df2)] = lst


print(df2.tail(20))
df2.to_csv("res.csv")

    codigoPostal                              variable dimension value_type  \
136        80060      SI_EF_TILTED_SURFACE_OPTIMAL_ANG       JUL    numeric   
137        80060      SI_EF_TILTED_SURFACE_OPTIMAL_ANG       AUG    numeric   
138        80060      SI_EF_TILTED_SURFACE_OPTIMAL_ANG       SEP    numeric   
139        80060      SI_EF_TILTED_SURFACE_OPTIMAL_ANG       OCT    numeric   
140        80060      SI_EF_TILTED_SURFACE_OPTIMAL_ANG       NOV    numeric   
141        80060      SI_EF_TILTED_SURFACE_OPTIMAL_ANG       DEC    numeric   
142        80060      SI_EF_TILTED_SURFACE_OPTIMAL_ANG       ANN    numeric   
143        80060  SI_EF_TILTED_SURFACE_OPTIMAL_ANG_ORT       JAN    string    
144        80060  SI_EF_TILTED_SURFACE_OPTIMAL_ANG_ORT       FEB    string    
145        80060  SI_EF_TILTED_SURFACE_OPTIMAL_ANG_ORT       MAR    string    
146        80060  SI_EF_TILTED_SURFACE_OPTIMAL_ANG_ORT       APR    string    
147        80060  SI_EF_TILTED_SURFACE_OPTIMAL_ANG_O

: 